# 05 — Opportunity Scoring & Ranking

**Objective:** Combine all signals into a ranked opportunity table.

## Opportunity Score Formula
```
opportunity = w1 * nutritional_gap
            + w2 * (1 - brand_concentration)
            + w3 * category_size
            + w4 * reformulation_feasibility
            + w5 * (1 - PL_saturation)
            + w6 * price_gap_margin
```

## Sensitivity Analysis
Vary weights and show ranking stability across 1000 random weight combinations.

In [ ]:
from src.analysis.opportunity_scorer import (
    compute_opportunity_score,
    sensitivity_analysis,
    normalise_column,
)
from src.visualisation.opportunity_matrix import plot_opportunity_quadrant

In [ ]:
# TODO: Build opportunity scores and run sensitivity analysis